Process Prediction
==
 - Load Data
 - Categorize / Normalize / Fillmissing
 - Create Datastructure for language model

# Imports

In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

In [2]:
from exp.eventlog import *

In [3]:
from exp.dl_utils import *

In [4]:
import editdistance as ed

# Load Data

In [5]:
log=import_xes(untar_data(URLs.BPIC_2012))

In [6]:
log.traceAttributes

,REG_DATE,AMOUNT_REQ
concept:name,,
173688,2011-10-01 00:38:44.546000+02:00,20000
173691,2011-10-01 08:08:58.256000+02:00,5000
173694,2011-10-01 08:10:30.287000+02:00,7000
173697,2011-10-01 08:11:08.865000+02:00,15000
173700,2011-10-01 08:15:39.894000+02:00,5000
173703,2011-10-01 09:45:25.806000+02:00,13500
173706,2011-10-01 09:45:37.274000+02:00,18000
173709,2011-10-01 09:57:42.994000+02:00,11000
173712,2011-10-01 09:58:30.533000+02:00,30000


In [7]:
log.events

,trace_id,event_id,org:resource,lifecycle:transition,concept:name,time:timestamp
0,173688,0,112,COMPLETE,A_SUBMITTED,2011-09-30 22:38:44.546000+00:00
1,173688,1,112,COMPLETE,A_PARTLYSUBMITTED,2011-09-30 22:38:44.880000+00:00
2,173688,2,112,COMPLETE,A_PREACCEPTED,2011-09-30 22:39:37.906000+00:00
3,173688,3,112,SCHEDULE,W_Completeren aanvraag,2011-09-30 22:39:38.875000+00:00
4,173688,4,None,START,W_Completeren aanvraag,2011-10-01 09:36:46.437000+00:00
5,173688,5,10862,COMPLETE,A_ACCEPTED,2011-10-01 09:42:43.308000+00:00
6,173688,6,10862,COMPLETE,O_SELECTED,2011-10-01 09:45:09.243000+00:00
7,173688,7,10862,COMPLETE,A_FINALIZED,2011-10-01 09:45:09.243000+00:00
8,173688,8,10862,COMPLETE,O_CREATED,2011-10-01 09:45:11.197000+00:00
9,173688,9,10862,COMPLETE,O_SENT,2011-10-01 09:45:11.380000+00:00


In [8]:
df=log.traceAttributes

# Data Processing

In [9]:
cat_columns=['event_id','org:resource','lifecycle:transition','concept:name',]
date_columns=['time:timestamp','REG_DATE']
num_columns=['AMOUNT_REQ']

In [12]:
def add_datepart(df, fldname, drop=True, time=False,utc=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, utc=utc,infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    cols=[]
    for n in attr: 
        col_name=targ_pre +"_"+ n
        df[col_name] = getattr(fld.dt, n.lower())
        cols.append(col_name)
    df[targ_pre + '_Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)
    return cols,targ_pre + '_Elapsed'

In [13]:
dp=log.traceAttributes
cols=add_datepart(dp,'REG_DATE',utc=True)
print(cols)
dp

(['REG_DATE_Year', 'REG_DATE_Month', 'REG_DATE_Week', 'REG_DATE_Day', 'REG_DATE_Dayofweek', 'REG_DATE_Dayofyear', 'REG_DATE_Is_month_end', 'REG_DATE_Is_month_start', 'REG_DATE_Is_quarter_end', 'REG_DATE_Is_quarter_start', 'REG_DATE_Is_year_end', 'REG_DATE_Is_year_start'], 'REG_DATE_Elapsed')


,AMOUNT_REQ,REG_DATE_Year,REG_DATE_Month,REG_DATE_Week,REG_DATE_Day,REG_DATE_Dayofweek,REG_DATE_Dayofyear,REG_DATE_Is_month_end,REG_DATE_Is_month_start,REG_DATE_Is_quarter_end,REG_DATE_Is_quarter_start,REG_DATE_Is_year_end,REG_DATE_Is_year_start,REG_DATE_Elapsed
concept:name,,,,,,,,,,,,,,
173688,20000,2011,9,39,30,4,273,True,False,True,False,False,False,1317422324
173691,5000,2011,10,39,1,5,274,False,True,False,True,False,False,1317449338
173694,7000,2011,10,39,1,5,274,False,True,False,True,False,False,1317449430
173697,15000,2011,10,39,1,5,274,False,True,False,True,False,False,1317449468
173700,5000,2011,10,39,1,5,274,False,True,False,True,False,False,1317449739
173703,13500,2011,10,39,1,5,274,False,True,False,True,False,False,1317455125
173706,18000,2011,10,39,1,5,274,False,True,False,True,False,False,1317455137
173709,11000,2011,10,39,1,5,274,False,True,False,True,False,False,1317455862
173712,30000,2011,10,39,1,5,274,False,True,False,True,False,False,1317455910


In [14]:
UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ = "xxunk xxpad xxbos xxeos xxrep xxwrep xxup xxmaj".split()
default_spec_tok = [UNK, PAD, BOS, EOS, TK_REP, TK_WREP, TK_UP, TK_MAJ]

from collections import OrderedDict

def uniqueify(x, sort=False):
    res = list(OrderedDict.fromkeys(x).keys())
    if sort: res.sort()
    return res

class Processor():
    def process(self, items): return items

class CategoryProcessor(Processor):
    def __init__(self,default_token=None): 
        self.vocab=None
        self.default_token=default_token

    def __call__(self, items):
        #The vocab is defined on the first use.
        if self.vocab is None:
            self.vocab = uniqueify(items)
            if self.default_token is not None:
                for o in reversed(self.default_token):
                    if o in self.vocab: self.vocab.remove(o)
                    self.vocab.insert(0, o)
            self.otoi  = {v:k for k,v in enumerate(self.vocab)}
        return [self.proc1(o) for o in items]
    def proc1(self, item):  return self.otoi[item]

    def deprocess(self, idxs):
        assert self.vocab is not None
        return [self.deproc1(idx) for idx in idxs]
    def deproc1(self, idx): return self.vocab[idx]


In [15]:
def encode_data(events,trace_attributes,cat_columns,date_columns,num_columns):
    vocab={}
    events=events.copy()
    trace_attributes=trace_attributes.copy()
    for d in date_columns:
        if d in list(events): 
            cat,con=add_datepart(events,d,utc=True)
            cat_columns+=cat
            num_columns.append(con)
        if d in list(trace_attributes): 
            cat,con=add_datepart(trace_attributes,d,utc=True)
            cat_columns+=cat
            num_columns.append(con)
    for c in cat_columns:
        if c in list(events): 
            proc=CategoryProcessor(default_spec_tok)
            events[c]=proc(events[c])
        if c in list(trace_attributes): 
            proc=CategoryProcessor()
            trace_attributes[c]=proc(trace_attributes[c])
        vocab[c]=proc   
    for n in num_columns:
        if n in list(events): 
            events[n]= events[n].astype(float)
            mean,std = events[n].mean(),events[n].std()
            events[n] = (events[n]-mean) / (1e-7 + std)
        if n in list(trace_attributes): 
            trace_attributes[n]= trace_attributes[n].astype(float)
            mean,std = trace_attributes[n].mean(),trace_attributes[n].std()
            trace_attributes[n] = (trace_attributes[n]-mean) / (1e-7 + std)
        vocab[n]=mean,std
    return events,trace_attributes,cat_columns,num_columns,vocab

events,trace_attributes,cat_columns,num_columns,vocabs=encode_data(log.events,log.traceAttributes,cat_columns,date_columns,num_columns)

In [ ]:
def encode_data(events, trace_attributes, cat_columns, date_columns, num_columns, vocab=None):
    if vocab is None:
        vocab = {}
        
    events = events.copy()
    trace_attributes = trace_attributes.copy()
    
    for d in date_columns:
        if d in list(events): 
            cat, con = add_datepart(events,d,utc=True)
            for i in range(len(cat)):
                if cat[i] not in cat_columns:
                    cat_columns.append(cat[i])
            if con not in num_columns:
                num_columns.append(con)
                
        if d in list(trace_attributes): 
            cat, con = add_datepart(trace_attributes,d,utc=True)
            for i in range(len(cat)):
                if cat[i] not in cat_columns:
                    cat_columns.append(cat[i])
            if con not in num_columns:
                num_columns.append(con)            
        
    for c in cat_columns: 
        if c in vocab.keys():
            proc = vocab[c]
            
            if c in list(events):
                events[c] = proc(events[c])
            if c in list(trace_attributes):
                trace_attributes[c] = proc(trace_attributes[c])
            
        else:
            if c in list(events): 
                proc = CategoryProcessor(default_spec_tok)
                events[c]=proc(events[c])
            if c in list(trace_attributes): 
                proc = CategoryProcessor()
                trace_attributes[c] = proc(trace_attributes[c])

            vocab[c] = proc   
        
    for n in num_columns:
        if n in vocab:
            mean, std = vocab[n]
            
            if n in list(events):
                events[n] = events[n].astype(float)
                events[n] = (events[n]-mean) / (1e-7 + std)
            if n in list(trace_attributes):
                trace_attributes[n] = trace_attributes[n].astype(float)
                trace_attributes[n] = (trace_attributes[n]-mean) / (1e-7 + std)
                
        else:
            if n in list(events): 
                events[n] = events[n].astype(float)
                mean, std = events[n].mean(), events[n].std()
                events[n] = (events[n]-mean) / (1e-7 + std)
            if n in list(trace_attributes): 
                trace_attributes[n] = trace_attributes[n].astype(float)
                mean, std = trace_attributes[n].mean(), trace_attributes[n].std()
                trace_attributes[n] = (trace_attributes[n]-mean) / (1e-7 + std)

            vocab[n] = mean, std
        
    return events, trace_attributes, cat_columns, num_columns, vocab

# Create Traces

In [ ]:
def create_traces(event_df,traceAttributes,trace_id='trace_id'):
    ll=[]
    trace_ids=[]
    cols=list(event_df)
    cols.remove(trace_id)
    for n, g in event_df.groupby(trace_id):
        l=[]
        
        for c in cols:
            l.append(list(g[c]))
        ll.append(l)
        trace_ids.append(n)  
        

    df=pd.DataFrame(ll,columns=cols)
    df.index=trace_ids
    df=pd.merge(df,traceAttributes,right_index=True, left_index=True)
    return df
traces=create_traces(events,trace_attributes)
traces

# Split in Train, Valid, Test

In [ ]:
def random_split(n,train=0.8,valid=0.1):
    shuffled=np.random.permutation(n)
    split1=int(n*train)
    split2=int(n*(train+valid))
    return shuffled[:split1],shuffled[split1:split2],shuffled[split2:]

In [ ]:
train_idx,valid_idx,test_idx=random_split(len(traces),train=0.7,valid=0.15)

In [ ]:
len(train_idx),len(valid_idx),len(test_idx),len(train_idx)+len(valid_idx)+len(test_idx)

In [ ]:
train,valid,test=traces.iloc[train_idx],traces.iloc[valid_idx],traces.iloc[test_idx]

# Complete Processing

In [ ]:


# encode data and create vocab
cat_columns=['event_id','org:resource','lifecycle:transition','concept:name',]
date_columns=['time:timestamp','REG_DATE']
num_columns=['AMOUNT_REQ']



events_enc,traceAttributes_enc,cat_columns,num_columns,vocabs=encode_data(log.events,log.traceAttributes,cat_columns,date_columns,num_columns)


In [ ]:
# create traces unencoded and encoded
traces=create_traces(log.events,log.traceAttributes)
traces_enc=create_traces(events_enc,traceAttributes_enc)



In [ ]:
# create train_idx,valid_idx,test_idx 
train_idx,valid_idx,test_idx = random_split(len(traces))

In [ ]:
# create train,valid,test 
_,_,test_orig=traces.iloc[train_idx],traces.iloc[valid_idx],traces.iloc[test_idx]
train,valid,test=traces_enc.iloc[train_idx],traces_enc.iloc[valid_idx],traces_enc.iloc[test_idx]

# Language Model DataLoader

In [ ]:
class PPDataSet():
    def __init__(self, df, bs=64, bptt=70, shuffle=False):
        self.bs,self.bptt,self.shuffle = bs,bptt,shuffle
        self.cols=list(df)

        total_len = sum(df.apply(lambda x: max([len(listify(x[k])) for k in self.cols]),axis=1))
        self.n_batch = total_len // self.bs

        self.batched=self.batchify(df)
        #print(self.bs,self.bptt,self.shuffle,total_len, self.n_batch)
        #print(self.batched)
    
    def __len__(self): return ((self.n_batch-1) // self.bptt) * self.bs
    
    def __getitem__(self, idx):
        source = self.batched[:,idx % self.bs]
        seq_idx = (idx // self.bs) * self.bptt
        x,y=source[:,seq_idx:seq_idx+self.bptt],source[:,seq_idx+1:seq_idx+self.bptt+1]
        return x,y
    
    def batchify(self,df):
        if self.shuffle: df=df.sample(frac=1)
        
        dd={}
        for c in self.cols:
            dd[c]=[]
        for i, row in df.iterrows():
            l=max([len(listify(row[c])) for c in self.cols])
            for c in self.cols:
                dd[c].append(tensor(row[c]).expand(l))
        for c in self.cols:
            s= torch.cat([torch.cat((tensor([2.0]),t.float(),tensor([3.0]))) for t in dd[c]])
            dd[c]=s[:self.n_batch * self.bs].view(self.bs, self.n_batch)
        return torch.stack([dd[c] for c in self.cols])
    
ppd=PPDataSet(train, shuffle=True,bs=64,bptt=70)
dl=DataLoader(ppd, batch_size=64)

In [ ]:
ppd.batched.shape,ppd.n_batch

In [ ]:
ppd_0=ppd[0]
ppd_0[0].shape,ppd_0[1].shape

In [ ]:
ppd_0[0][1]

In [ ]:
iter_dl = iter(dl)
xb,yb = next(iter_dl)
xb.size()

In [ ]:
xb[0][0]

In [ ]:
list(vocabs.keys())

In [ ]:
list(traces_enc)

In [ ]:
list(vocabs.keys())

In [ ]:
index=3
v=vocabs[list(vocabs.keys())[index]]
print(list(vocabs.keys())[index])
" ".join([str(x) for x in v.deprocess(xb[0][index].int().flatten())])

# Basic Model

In [ ]:
class Model(nn.Module):
    def __init__(self, n_in,n_out,n_emb,nh):
        super().__init__()
        self.emb=nn.Embedding(n_in, 7, padding_idx=1)
        self.lin1=nn.Linear(7,nh)
        self.relu=nn.ReLU()
        self.lin2=nn.Linear(nh,n_out)
        
    def __call__(self, x):
        x=x[:,3] # magic number for 'concept:name'
        x=x.long()
        x=self.emb(x)
        x=self.lin1(x)
        x=self.relu(x)
        x=self.lin2(x)
        return x.float()

In [ ]:
def getModel():
    bs,bptm,vocab=64,70,len((vocabs['concept:name']).vocab) # Stupid 'concept:name' model
    n_emb,nh=int(vocab/2),10
    model=Model(bs*bptm,vocab,n_emb,nh)
    return model

In [ ]:
len((vocabs['event_id']).vocab)

In [ ]:
model=getModel()
pred = model(xb)
pred.shape,yb[:,0].shape

In [ ]:
def cross_entropy_activity(input, target):
    target=target[:,3] # magic number for 'concept:name'
    bs,sl =target.size()
    return F.cross_entropy(input.view(bs * sl, -1), target.flatten().long())
cross_entropy_activity(pred,yb)

In [ ]:
def accuracy_activity(input, target): 
    target=target[:,3] # magic number for 'concept:name'
    bs,sl =target.size()
    return (torch.argmax(input.view(bs * sl, -1), dim=1)==target.flatten().long()).float().mean()
accuracy_activity(pred,yb)

# Training Loop

**DATA**

In [ ]:
def get_dls(train_ds, valid_ds, bs, **kwargs):
    return (DataLoader(PPDataSet(train_ds, shuffle=True), batch_size=bs),
            DataLoader(PPDataSet(valid_ds, shuffle=False), batch_size=bs))

In [ ]:
data = DataBunch(*get_dls(train, valid, 64))

**Callbacks**

In [ ]:
sched = combine_scheds([0.3, 0.7], [sched_cos(0.3, 0.6), sched_cos(0.6, 0.2)]) 

In [ ]:
class CudaCallback(Callback):
    def begin_fit(self): self.model.cuda()
    def begin_batch(self): self.run.xb,self.run.yb = self.xb.cuda(),self.yb.cuda()

In [ ]:
cbfs = [partial(AvgStatsCallback,accuracy_activity),
        CudaCallback, 
        Recorder,
        partial(ParamScheduler, 'lr', sched),
        ProgressBarCallback,
       ]

**Model**

In [ ]:
opt_func = partial(Optimizer, steppers=[sgd_step])

In [ ]:
model=getModel()

In [ ]:
opt = opt_func(model.parameters(), lr=0.5)


**Learner**

In [ ]:
learn = Learner(model,data,cross_entropy_activity,cb_funcs=cbfs,opt_func=opt_func)


In [ ]:
learn.fit(20) 

In [ ]:
basic_model=learn.model
basic_model(xb.cuda()).shape

# AWD-LSTM

In [ ]:
#export
def dropout_mask(x, sz, p):
    return x.new(*sz).bernoulli_(1-p).div_(1-p)

In [ ]:
x = torch.randn(10,10)
mask = dropout_mask(x, (10,10), 0.5); mask

In [ ]:
(x*mask).std(),x.std()

In [ ]:
#export
class RNNDropout(nn.Module):
    def __init__(self, p=0.5):
        super().__init__()
        self.p=p

    def forward(self, x):
        if not self.training or self.p == 0.: return x
        m = dropout_mask(x.data, (x.size(0), 1, x.size(2)), self.p)
        return x * m

In [ ]:
dp = RNNDropout(0.3)
tst_input = torch.randn(3,3,7)
tst_input.shape, dp(tst_input).shape

In [ ]:
#export
import warnings

WEIGHT_HH = 'weight_hh_l0'

class WeightDropout(nn.Module):
    def __init__(self, module, weight_p=[0.], layer_names=[WEIGHT_HH]):
        super().__init__()
        self.module,self.weight_p,self.layer_names = module,weight_p,layer_names
        for layer in self.layer_names:
            #Makes a copy of the weights of the selected layers.
            w = getattr(self.module, layer)
            self.register_parameter(f'{layer}_raw', nn.Parameter(w.data))
            self.module._parameters[layer] = F.dropout(w, p=self.weight_p, training=False)

    def _setweights(self):
        for layer in self.layer_names:
            raw_w = getattr(self, f'{layer}_raw')
            self.module._parameters[layer] = F.dropout(raw_w, p=self.weight_p, training=self.training)

    def forward(self, *args):
        self._setweights()
        with warnings.catch_warnings():
            #To avoid the warning that comes because the weights aren't flattened.
            warnings.simplefilter("ignore")
            return self.module.forward(*args)

In [ ]:
module = nn.LSTM(5, 2)
dp_module = WeightDropout(module, 0.4)
getattr(dp_module.module, WEIGHT_HH)

In [ ]:
tst_input = torch.randn(4,20,5)
h = (torch.zeros(1,20,2), torch.zeros(1,20,2))
x,h = dp_module(tst_input,h)
getattr(dp_module.module, WEIGHT_HH)

In [ ]:
#export
class EmbeddingDropout(nn.Module):
    "Applies dropout in the embedding layer by zeroing out some elements of the embedding vector."
    def __init__(self, emb, embed_p):
        super().__init__()
        self.emb,self.embed_p = emb,embed_p
        self.pad_idx = self.emb.padding_idx
        if self.pad_idx is None: self.pad_idx = -1

    def forward(self, words, scale=None):
        if self.training and self.embed_p != 0:
            size = (self.emb.weight.size(0),1)
            mask = dropout_mask(self.emb.weight.data, size, self.embed_p)
            masked_embed = self.emb.weight * mask
        else: masked_embed = self.emb.weight
        if scale: masked_embed.mul_(scale)
        return F.embedding(words, masked_embed, self.pad_idx, self.emb.max_norm,
                           self.emb.norm_type, self.emb.scale_grad_by_freq, self.emb.sparse)

In [ ]:
enc = nn.Embedding(100, 7, padding_idx=1)
enc_dp = EmbeddingDropout(enc, 0.5)
tst_input = torch.randint(0,100,(8,))
enc_dp(tst_input)

In [ ]:
#export
def to_detach(h):
    "Detaches `h` from its history."
    return h.detach() if type(h) == torch.Tensor else tuple(to_detach(v) for v in h)

In [ ]:
#export
class AWD_LSTM(nn.Module):
    "AWD-LSTM inspired by https://arxiv.org/abs/1708.02182."
    initrange=0.1

    def __init__(self, vocab_sz, emb_sz, n_hid, n_layers, pad_token,
                 hidden_p=0.2, input_p=0.6, embed_p=0.1, weight_p=0.5):
        super().__init__()
        self.bs,self.emb_sz,self.n_hid,self.n_layers = 1,emb_sz,n_hid,n_layers
        self.emb = nn.Embedding(vocab_sz, emb_sz, padding_idx=pad_token)
        self.emb_dp = EmbeddingDropout(self.emb, embed_p)
        self.rnns = [nn.LSTM(emb_sz if l == 0 else n_hid, (n_hid if l != n_layers - 1 else emb_sz), 1,
                             batch_first=True) for l in range(n_layers)]
        self.rnns = nn.ModuleList([WeightDropout(rnn, weight_p) for rnn in self.rnns])
        self.emb.weight.data.uniform_(-self.initrange, self.initrange)
        self.input_dp = RNNDropout(input_p)
        self.hidden_dps = nn.ModuleList([RNNDropout(hidden_p) for l in range(n_layers)])

    def forward(self, input):
        bs,sl = input.size()
        if bs!=self.bs:
            self.bs=bs
            self.reset()
        raw_output = self.input_dp(self.emb_dp(input))
        new_hidden,raw_outputs,outputs = [],[],[]
        for l, (rnn,hid_dp) in enumerate(zip(self.rnns, self.hidden_dps)):
            raw_output, new_h = rnn(raw_output, self.hidden[l])
            new_hidden.append(new_h)
            raw_outputs.append(raw_output)
            if l != self.n_layers - 1: raw_output = hid_dp(raw_output)
            outputs.append(raw_output) 
        self.hidden = to_detach(new_hidden)
        return raw_outputs, outputs

    def _one_hidden(self, l):
        "Return one hidden state."
        nh = self.n_hid if l != self.n_layers - 1 else self.emb_sz
        return next(self.parameters()).new(1, self.bs, nh).zero_()

    def reset(self):
        "Reset the hidden states."
        self.hidden = [(self._one_hidden(l), self._one_hidden(l)) for l in range(self.n_layers)]

In [ ]:
#export
class LinearDecoder(nn.Module):
    def __init__(self, n_out, n_hid, output_p, tie_encoder=None, bias=True):
        super().__init__()
        self.output_dp = RNNDropout(output_p)
        self.decoder = nn.Linear(n_hid, n_out, bias=bias)
        if bias: self.decoder.bias.data.zero_()
        if tie_encoder: self.decoder.weight = tie_encoder.weight
        else: init.kaiming_uniform_(self.decoder.weight)

    def forward(self, input):
        raw_outputs, outputs = input
        output = self.output_dp(outputs[-1]).contiguous()
        decoded = self.decoder(output.view(output.size(0)*output.size(1), output.size(2)))
        return decoded, raw_outputs, outputs

In [ ]:
#export
class SequentialRNN(nn.Sequential):
    "A sequential module that passes the reset call to its children."
    def reset(self):
        for c in self.children():
            if hasattr(c, 'reset'): c.reset()

In [ ]:
#export
def get_language_model(vocab_sz, emb_sz, n_hid, n_layers, pad_token, output_p=0.4, hidden_p=0.2, input_p=0.6, 
                       embed_p=0.1, weight_p=0.5, tie_weights=True, bias=True):
    rnn_enc = AWD_LSTM(vocab_sz, emb_sz, n_hid=n_hid, n_layers=n_layers, pad_token=pad_token,
                       hidden_p=hidden_p, input_p=input_p, embed_p=embed_p, weight_p=weight_p)
    enc = rnn_enc.emb if tie_weights else None
    return SequentialRNN(rnn_enc, LinearDecoder(vocab_sz, emb_sz, output_p, tie_encoder=enc, bias=bias))

In [1]:
tok_pad = v.index(PAD)

NameError: name 'v' is not defined

In [ ]:
v

In [ ]:
tst_model = get_language_model(len(v), 300, 300, 2, tok_pad)
tst_model = tst_model

In [ ]:
model

In [ ]:
tst_model

In [ ]:
z=tst_model(xb[0])
z

In [ ]:
z[0].shape,64*70

In [ ]:
def cross_entropy_flat(input, target):
    bs,sl = target.size()
    return F.cross_entropy(input.view(bs * sl, -1), target.view(bs * sl))

In [ ]:
def accuracy_flat(input, target):
    bs,sl = target.size()
    return accuracy(input.view(bs * sl, -1), target.view(bs * sl))

In [ ]:
decoded, raw_outputs, outputs = z

In [ ]:
xb[0].size(),decoded.size()

In [ ]:
accuracy_flat(decoded,yb[0])

In [ ]:
cross_entropy_flat(decoded,yb[0])

In [ ]:
bs,sl = yb[0].size()
bs,sl

In [ ]:
len(yb[0].view(bs * sl))

In [ ]:
decoded

In [ ]:
(torch.argmax(decoded, dim=1)==yb[0].view(bs * sl).cuda()).float().mean()

In [ ]:
decoded.shape

In [ ]:
len(raw_outputs),len(outputs)

In [ ]:
[o.size() for o in raw_outputs], [o.size() for o in outputs]

# Training Loop with AWD-LSTM

In [ ]:
data = DataBunch(*get_dls(train, valid, 64))

In [ ]:
#export
class GradientClipping(Callback):
    def __init__(self, clip=None): self.clip = clip
    def after_backward(self):
        if self.clip:  nn.utils.clip_grad_norm_(self.run.model.parameters(), self.clip)

In [ ]:
#export
class RNNTrainer(Callback):
    def __init__(self, α, β): self.α,self.β = α,β
    
    def after_pred(self):
        #Save the extra outputs for later and only returns the true output.
        self.raw_out,self.out = self.pred[1],self.pred[2]
        self.run.pred = self.pred[0]
    
    def after_loss(self):
        #AR and TAR
        if self.α != 0.:  self.run.loss += self.α * self.out[-1].float().pow(2).mean()
        if self.β != 0.:
            h = self.raw_out[-1]
            if len(h)>1: self.run.loss += self.β * (h[:,1:] - h[:,:-1]).float().pow(2).mean()
                
    def begin_epoch(self):
        pass
        #Shuffle the texts at the beginning of the epoch
        #if hasattr(self.dl.dataset, "batchify"): self.dl.dataset.batchify()

In [ ]:
emb_sz, nh, nl = 300, 300, 2
model = get_language_model(len(v), emb_sz, nh, nl, tok_pad, input_p=0.6, output_p=0.4, weight_p=0.5, 
                           embed_p=0.1, hidden_p=0.2)

In [ ]:
cbs = [partial(AvgStatsCallback,accuracy_flat),
       CudaCallback, Recorder,
       partial(GradientClipping, clip=0.1),
       partial(RNNTrainer, α=2., β=1.),
       ProgressBarCallback]

In [ ]:
learn = Learner(model, data, cross_entropy_flat, lr=5e-3, cb_funcs=cbs, opt_func=adam_opt())

In [ ]:
learn.fit(20)

# Testing
nach Learning Accurate LSTM Models of Business Processes ->BPM17, 2019

In [ ]:
class BasicModel():
    
    def __init__(self, model, test_df, vocab):
        self.model = model
        self.test_df = test_df
        self.vocab = vocab
    
    def predict(self, traces):
        # since traces is a list of indeces the first thing we need to do is get the actual data
        # does iloc make sense in this case? 
        data = test_df.iloc[traces]

        # now we will encode the test_data the same way we encoded the training data
        # --> that's why we're using the vocab of the training data in our encode function
        events_enc, traceAttributes_enc, cat_columns, num_columns, test_vocab = encode(log.events, log.trace_attributes, cat_columns, date_columns, num_columns, vocab=vocab)
        
        # model auf encodeten daten anwenden -> siehe sample methode, solange predicten bis endstring=xxeos
        pass

In [ ]:
class PerfectModel():
    def __init__(self,groundtruth):
        self.groundtruth=groundtruth
    def predict(self,traces):
        return self.groundtruth

In [ ]:
class StupidNextStepModel():
    def predict(self,traces):
        return 'A_DECLINED'

In [ ]:
list(test)

In [ ]:
def process_data_for_suffix_prediction(test,cols=None,startIndex=1):
    x,y={},{}
    if cols == None: cols=list(test)
    cols=listify(cols)
    for col in cols:
        x[col],y[col]=[],[]
        for trace in test[col]: // test.index statt test[col]
            for i in range(startIndex,len(listify(trace))):
                x[col].append(trace[:i])
                y[col].append(trace[i:])
    return x,y
x,y=process_data_for_suffix_prediction(test)
x.keys()

In [ ]:
def process_data_for_next_step_prediction(test,col=None,startIndex=1):
    x,y=[],[]
    traces=test.values
    if col!=None: traces=test[col].values
    for trace in traces:
        for i in range(startIndex,len(trace)):
            x.append(flatten_ir_list(listify(trace[:i])))
            y.append(flatten_ir_list(listify(trace[i])))
    return x,y
x,y=process_data_for_next_step_prediction(test)
(list(zip(x,y)))[1]

In [ ]:
test.values[0]

In [ ]:
len(x)

In [ ]:
def suffix_measure(preds,ys,col='concept:name'):
    sum=0.0
    for p,y in zip(preds[col],ys[col]):
        l=len(p)
        d=ed.eval(p,y)
        sim=1-d/l
        sum+=sim
    return sum/len(preds[col])
x,y=process_data_for_suffix_prediction(test)
model=PerfectModel(y)
preds=model.predict(x)
suffix_measure(preds,y)

In [ ]:
preds.keys()

In [ ]:
def next_step_measure(preds,ys):
    # Simple accuracy measure
    # Do I have to weight it? Check Paper!
    return (np.array(preds)==np.array(ys)).astype(float).mean()
x,y=process_data_for_next_step_prediction(test)
model=PerfectModel(y)
preds=model.predict(x)
next_step_measure(preds,y)

In [ ]:
model=StupidNextStepModel()
preds=model.predict(x)
next_step_measure(preds,y)

In [ ]:
model=BasicModel(basic_model,test,vocab)
x,y=process_data_for_next_step_prediction(test)
preds=model.predict(x)
next_step_measure(preds,y)

# Export

In [ ]:
! /home/lahann/anaconda3/envs/fastpm/bin/python notebook2script.py 02_fastpm_process_prediction.ipynb